<a href="https://colab.research.google.com/github/thienluc2017/BERT_19021324/blob/main/BERT_19021324.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MSV:19021324.

Họ và tên :Lê Thiên Lực



# A - Introduction
Overview
BERT, or Bidirectional Embedding Representations from Transformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks.
The academic paper can be found here: https://arxiv.org/abs/1810.04805.

**Reference**:

To understand BERT and learn how to implement BERT, I highly recommend reading the following sources:

- Good explanation and visualization of BERT: [https://jalammar.github.io/illustrated-bert/](https://jalammar.github.io/illustrated-bert/)
- How to finetune BERT models: https://towardsdatascience.com/how-to-train-bert-aaad00533168
- Finetune notebook from HuggingFace: [https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/training.ipynb#scrollTo=R8NJ_RtIUz4l](https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/training.ipynb#scrollTo=R8NJ_RtIUz4l)
- Implementation code(write by my UET mentor: Phi Hung Nguyen) : [https://colab.research.google.com/drive/1bBnyzCkj2Imdk5hrZ0OAOH1l8aDLF4YW?usp=sharing](https://colab.research.google.com/drive/1wdt7z8UcDla3EAjJXI-pxmqZEEB3ry4Z?usp=sharing)



# B - Setup


## 1. Install dependences and load Essential Libraries
- pytorch-lightning: a simple trainer to help you minize code base
- transformers: library contains multiple BERT models
- sentencepiece: a word-to-vect library with fast implementation

In [ ]:
!pip install pytorch-lightning
!pip install tensorflow==1.15
!pip install transformers
!pip install sentencepiece


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Load data

In [ ]:
DATA_ROOT_DIR="/content/drive/MyDrive/shopee-sentiment"
!ls $DATA_ROOT_DIR

sample_submission.csv  train.csv		       train.xlsx
test.csv	       train_preprocess.csv
test_preprocess.csv    train_preprocess_unsegment.csv


In [ ]:
# include some dependence
import pandas as pd
import numpy as np
from torch.utils.data import random_split, DataLoader, Dataset
import pytorch_lightning as pl
import torch.nn as nn
import torch
#
DATA_DIR = DATA_ROOT_DIR + '/train_preprocess_unsegment.csv'

In [ ]:
# Use pandas to read csv, this will return a excel like table data
train = pd.read_csv(DATA_DIR, encoding='utf-8')
train.sample(5)

,Unnamed: 0,id,text,class,preprocess_text
19018,19018,02dd8a6a,Cuối tuần hẹn_hò người_yêu đến CGV là hợp_lí n...,1,cuối tuần hẹn hò người yêu đến cgv là hợp lí n...
5919,5919,4934dfc5,Nghe nhân_viên giới_thiệu khúc bạch ngon lắm n...,1,nghe nhân viên giới thiệu khúc bạch ngon lắm n...
23039,23039,0b0f0acc,Mình mua phần kem hủ gần 1 kg ở cửa_hàng này n...,1,mình mua phần kem hủ gần 1kg ở cửa hàng này nè...
24342,24342,9dcd0a53,Gọi đồ uống nóng thì nguội_ngắt . Gọi bánh thì...,0,gọi đồ uống nóng thì nguội ngắt gọi bánh thì c...
9813,9813,9ecffca1,đồ uống ở đây khá ngon và giá cũng vô_cùng hợp...,1,đồ uống ở đây khá ngon và giá cũng vô cùng hợp...


We will randomly split the entire training data into two sets: a train set with 80% of the data and a validation set with 20% of the data. We will perform hyperparameter tuning using cross-validation on the train set and use the validation set to compare models.

In [ ]:
from sklearn.model_selection import train_test_split

X = train.preprocess_text.values
y = train.id.values

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.2, random_state=2020)

### 2.3. Load Test Data
The test data contains 4555 examples with no label. About 300 examples are non-complaining tweets. Our task is to identify their `id` and examine manually whether our results are correct.

In [ ]:
DATA_DIR = DATA_ROOT_DIR + '/test.csv'
test_data = pd.read_csv(DATA_DIR, encoding='utf-8')
test_data.sample(5)

,id,text
828,7cdc42a5,"Được bạn bè rủ rê lên Phố Cổ chơi , lang thang..."
356,857c755e,Nhìn từ ngoài vào thấy khá ấn tượng trong cách...
260,6dc660b0,"Quán nhỏ , chính xác là xe bánh mì kê thêm mấy..."
1183,28773101,cơ sở này hơi khó tìm vì lối vào đầu đường đầy...
1260,935ab00c,Cái này hôm trước mình với bạn mình qua rồi nè...
